In [ ]:
import pandas as pd
from premise import NewDatabase
from brightway2 import projects, databases
from pyam import IamDataFrame

# CONFIGURATION
CSV_PATH = "DE_GRIDMIX_SCENARIO_02_PREDICTIONS.csv"
PROJECT_NAME = "DE_GRIDMIX"
SOURCE_DB_NAME = "ecoinvent 3.8 cutoff"
BIOSPHERE_NAME = "biosphere3"

# additional mappings for custom technologies (comment this out, if you want to simplify the first part of business, later on it's something we should have)
CUSTOM_SCENARIOS = pd.DataFrame([
    {
        "model": "custom",
        "pathway": "Base",
        "variable": "Electricity|Production|Hydrogen",
        "name": "market group for hydrogen",
        "unit": "kilowatt hour",
        "location": "DE"
    },
    {
        "model": "custom",
        "pathway": "Base",
        "variable": "Electricity|Production|Waste",
        "name": "electricity production from municipal solid waste, incineration",
        "unit": "kilowatt hour",
        "location": "DE"
    }
])
# ------ 

# Set or create Brightway2 project 
projects.set_current(PROJECT_NAME)

# Ensure biosphere3 database exists
if BIOSPHERE_NAME not in databases:
    from bw2io import create_default_biosphere3
    create_default_biosphere3()

# Check if the source database exists
# This is a 'manual' check if the database files exist
if SOURCE_DB_NAME not in databases:
    raise ValueError(f"Required source database '{SOURCE_DB_NAME}' not found in Brightway.")

# Load scenario CSV & convert to IamDataFrame 
df = pd.read_csv(CSV_PATH)
iam_df = IamDataFrame(df)

# Initialize Premise database 
db = NewDatabase(
    scenarios=[{
        "model": "remind",
        "pathway": "SSP2-Base",
        "year": 2030,
        "iam_data": iam_df
    }],
    source_db=SOURCE_DB_NAME,
    biosphere_name=BIOSPHERE_NAME,
    external_scenarios=[CUSTOM_SCENARIOS] # If commenting out the above scenarios, you have to also comment this out
)
